## Query header

In this notebook, we show several ways to query the fits headers with Gen3\
Craig Lage - 03-Jun-21

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
butler = butlerUtils.makeDefaultButler("LATISS")
instrument = 'LATISS'

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTCam")
instrument = 'LSSTCam'

In [ ]:
butler.get_known_repos()

In [ ]:
coll = butler.collections
for item in coll:
    print(item)

In [ ]:
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", "LSSTCam/runs/quickLook"])

In [ ]:
butler = Butler('LATISS', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/runs/quickLook"])

In [ ]:
collections = butler.registry.queryCollections()
for item in collections:
    print(item)

In [ ]:
registry = butler.registry
for dtype in registry.queryDatasetTypes()[:]:
    try:
        datasetRefs = list(registry.queryDatasets(datasetType=dtype,collections=butler.registry.queryCollections()))
        if len(datasetRefs) > 0:
            print(len(datasetRefs), dtype )
    except:
        print("Error", dtype )

In [ ]:
expId = 2025070600500
raw = butler.get('raw', detector=94, exposure=expId)

In [ ]:
from lsst.summit.utils.plotting import plot
expId = 2025041900287
calexp = butler.get('calexp', detector=150, visit=expId)
x=plot(calexp, stretch='log')
axs = x.get_axes()
axs[0].set_title(f"{expId} R33 S12")
plt.savefig(f"/home/cslage/DATA/{expId}_R33_S12.png")

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/runs/quickLook"])

In [ ]:
butler = Butler('/repo/embargo', collections=["LSSTComCamSim/quickLook/24"])

In [ ]:
butler = Butler('/repo/main', collections=["LSSTCam/raw/all", "LSSTCam/calib", "LSSTCam/runs/quickLook"])

In [ ]:
dayObs = 20251223
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)


In [ ]:
expId = 2025122300027
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])

In [ ]:
expIds = [2024101600421, 2025010700169, 2025010700170]
for expId in expIds:
    print(f"expId = {expId}")
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    for key in mData.keys():
        if "HVBIAS" in key:
            print(key, mData[key])

In [ ]:
expIds = [2024101600421, 2025010700169, 2025010700170, 2025010800025]
for expId in expIds:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    print(f"For {expId}, the number of header keys = {len(mData.keys())}")

In [ ]:
expIds = [2024101600421, 2025010700174]
mData1 = butler.get('raw.metadata', detector=0, exposure=expIds[0])
mData2 = butler.get('raw.metadata', detector=0, exposure=expIds[1])
n = 0
for key in mData1.keys():
    if not key in mData2.keys():
        n += 1
        print(key)


In [ ]:
record.observation_reason

In [ ]:
print("ExpId,            RA                   Dec")
for exp in range(2022101800025, 2022101800047):
    mData = butler.get('raw.metadata', detector=0, exposure=exp)
    print(exp, mData['RASTART'], mData['DECSTART'])
    

In [ ]:
expId = 2023110800415
raw = butler.get('raw', detector=0, exposure=expId)

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = raw.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=10000, vmax=30000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


In [ ]:
ycoord = 3500

plt.figure(figsize=(4,8))
plt.subplots_adjust(hspace=0.5)
plotCounter = 1
plt.suptitle("AuxTel darks with shorter exposure time.")
for expId in [2022110900030, 2022110900095, 2022110900150]:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    rawData = butler.get('raw', detector=0, exposure=expId)
    arr = rawData.image.array
    plt.subplot(3,1,plotCounter)
    plt.plot(arr[ycoord,:])
    plt.title(f"{expId}, Dark Exposure Time = {mData['EXPTIME']}, Ycoord = {ycoord}")
    plotCounter += 1

In [ ]:
exposureList[:][27]

In [ ]:
exposureList[0][1].fields

In [ ]:
from lsst.daf.butler import Butler


nTot=0
for [expId,record] in exposureList:
    nExp = 0
    for detector in range(9):
        try:
            mData = butler.get('raw.metadata', detector=detector, exposure=expId)
        except:
            nTot += 1
            nExp += 1
            #print(f"Detector {detector} from expId {expId} missing")
    if nExp != 0:
        print(f" expId {expId} is missing {nExp} files.")
print(f" Total of {n} exposures missing")

In [ ]:
expId1 = 2023070400115
expId2 = 2023042000200
mData1 = butler.get('raw.metadata', detector=0, exposure=expId1)
mData2 = butler.get('raw.metadata', detector=0, exposure=expId2)
for key in mData.keys():
    #if 'E' in key:
    print(key, mData1[key])#, mData2[key])

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/runs/quickLook"])
dataId = dict(day_obs=20240222, detector=0, seq_num=254)

dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
butler.collections

In [ ]:
butler.registry.queryDatasetTypes()

In [ ]:
expId = 2024022200254
exp = butler.get('raw', detector=0, exposure=expId)


In [ ]:
expId = 2024022200254
exp = butler.get('quickLookExp', detector=0, exposure=expId)


In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
mBias = butler.get('bias', detector=0, exposure=2022092900947)

In [ ]:
from pathlib import Path

In [ ]:
dir = "/home/craiglagegit/WORK/Notebook_Keeper/summit_notebooks"

## Snippet to test for ingestion - LATISS

In [ ]:
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultButler("LATISS")

dayObs = 20250324
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(f"{record.id}\t{record.observation_type}\t{record.exposure_time}\t{record.physical_filter}")


## Snippet to test for ingestion - LSSTCam

In [ ]:
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultButler("LSSTCam")

dayObs = 20250330
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(f"{record.id}\t{record.observation_type}\t{record.exposure_time}\t{record.physical_filter}")


In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
expId = 2025022500608
#expId = 2025022500202
#expId = 2024070800403
instrument='LATISS'
raw = butler.get('raw', detector=0, exposure=expId, instrument=instrument)

uri = butler.getURI('raw', dataId={'instrument': 'LATISS', 'detector':0, 'exposure':expId})
print(uri)